In [1]:
# importing modules 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder 
%matplotlib inline

In [2]:
df=pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# data preprocessing 
##grouping Cabin as per event rate
seg=df[['Cabin','Survived']].groupby(by='Cabin').mean()
seg.rename(columns={'Survived':'Cabin_segs'},inplace=True);
seg.reset_index(inplace=True)
df=df.merge(seg,on='Cabin',how='outer')
##dropping Name, ticket and Passenger ID and Cabin
cols_drop=['Name','Ticket','PassengerId','Cabin','Survived']
X= df.drop(cols_drop,axis=1)
y= df['Survived']
#train/val split #stratification 
from sklearn.model_selection import train_test_split 
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, test_size=0.2, stratify=y)
#missing value imputation
X_train['Age'].fillna(X_train['Age'].mean(),inplace=True)
X_val['Age'].fillna(X_train['Age'].mean(),inplace=True)

X_train['Cabin_segs'].fillna(X_train['Cabin_segs'].mean(),inplace=True)
X_val['Cabin_segs'].fillna(X_train['Cabin_segs'].mean(),inplace=True)

X_train['Embarked'].fillna(X_train.mode().loc[0,'Embarked'],inplace=True)
X_val['Embarked'].fillna(X_train.mode().loc[0,'Embarked'],inplace=True)
#standardisation 
from sklearn.preprocessing import StandardScaler
scalar=StandardScaler()
X_train[['Age','Fare']]=scalar.fit_transform(X_train[['Age','Fare']])
X_val[['Age','Fare']]=scalar.transform(X_val[['Age','Fare']])
#one hot encoding 
X_train=pd.get_dummies(X_train.copy())
X_val=pd.get_dummies(X_val.copy())

C:\Users\Smart\anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-3-ace8504d718f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[['Age','Fare']]=scalar.fit_transform(X_train[['Age','Fare']])
C:\Users\Smart\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [18]:
#tensorflow logit
alpha, epochs = 0.01, 50
m, n = X_train.shape 

X= tf.placeholder(tf.float32,[m,n])
Y= tf.placeholder(tf.float32,[m,2])

W= tf.Variable(tf.zeros([n,2]),name='weights')
b= tf.Variable(tf.zeros([m,2]),name='bias')

logits=tf.matmul(X,W)+b
entropy=tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,labels=Y)
loss=tf.reduce_mean(entropy)
optimizer=tf.train.GradientDescentOptimizer(learning_rate=
                                            alpha).minimize(loss)
init=tf.global_variables_initializer()

In [28]:
y_train

456    0
411    0
226    1
828    1
381    0
      ..
120    0
439    0
476    0
41     0
73     0
Name: Survived, Length: 712, dtype: int64

In [26]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(epochs):
        sess.run([optimizer,loss],feed_dict={X:X_train, Y:y_train})

ValueError: Cannot feed value of shape (712,) for Tensor 'Placeholder_3:0', which has shape '(712, 2)'